# Running Dask on the cluster with MLRun

```{admonition} Note
Dask is supported at the Tech Preview level only.
```

The Dask framework enables you to parallelize your Python code and run it as a distributed process on an Iguazio cluster and dramatically accelerate the performance. <br>
In this notebook you'll learn how to create a Dask cluster and then an MLRun function running as a Dask client. <br>
It also demonstrates how to run parallelize custom algorithm using Dask Delayed option.

For more information on Dask over Kubernetes: https://kubernetes.dask.org/en/latest/.

## Set up the environment

In [1]:
# set mlrun api path and artifact path for logging
import mlrun

project = mlrun.get_or_create_project("dask-demo", "./")

## Create and start Dask cluster
Dask functions can be local (local workers), or remote (use containers in the cluster). In the case of remote you 
can specify the number of replicas (optional) or leave blank for auto-scale.<br>
Use the `new_function()` to define the Dask cluster and set the desired configuration of that clustered function.

If the Dask workers need to access the shared file system, apply a shared volume mount (e.g. via v3io mount).

The Dask function spec has several unique attributes (in addition to the standard job attributes):

* **.remote** &mdash; bool, use local or clustered dask
* **.replicas** &mdash; number of desired replicas, keep 0 for auto-scale
* **.min_replicas**, **.max_replicas** &mdash; set replicas range for auto-scale
* **.scheduler_timeout** &mdash; cluster is killed after timeout (inactivity), default is '60 minutes'
* **.nthreads** &mdash; number of worker threads
<br>

If you want to access the Dask dashboard or scheduler from remote you need to use NodePort service type (set `.service_type` to 'NodePort'), and the external IP need to be specified in the MLRun configuration (mlconf.remote_host). This is set automatically if you are running on an Iguazio cluster.

Specify the kind (dask) and the container image:

In [2]:
# create an mlrun function that will init the dask cluster
dask_cluster_name = "dask-cluster"
dask_cluster = mlrun.new_function(dask_cluster_name, kind="dask", image="mlrun/mlrun")
dask_cluster.apply(mlrun.mount_v3io())

In [3]:
# set range for # of replicas with replicas and max_replicas
dask_cluster.spec.min_replicas = 1
dask_cluster.spec.max_replicas = 4

# set the use of dask remote cluster (distributed)
dask_cluster.spec.remote = True
dask_cluster.spec.service_type = "NodePort"

# set dask memory and cpu limits
dask_cluster.with_worker_requests(mem="2G", cpu="2")

### Initialize the Dask Cluster

When you request the dask cluster `client` attribute, it verifies that the cluster is up and running:

In [4]:
# init dask client and use the scheduler address as param in the following cell
dask_cluster.client

## Creating a function that runs over Dask

In [5]:
# mlrun: start-code

Import mlrun and dask. Nuclio is only used to convert the code into an MLRun function.

In [6]:
import mlrun

In [8]:
from dask.distributed import Client
from dask import delayed
from dask import dataframe as dd

import warnings
import numpy as np
import os
import mlrun

warnings.filterwarnings("ignore")

### Python function code

This simple function reads a .csv file using dask dataframe. It runs the `groupby` and `describe` functions on the dataset, and stores the results as a dataset artifact.<br>

In [9]:
def test_dask(
    context, dataset: mlrun.DataItem, client=None, dask_function: str = None
) -> None:
    # setup dask client from the MLRun dask cluster function
    if dask_function:
        client = mlrun.import_function(dask_function).client
    elif not client:
        client = Client()

    # load the dataitem as dask dataframe (dd)
    df = dataset.as_df(df_module=dd)

    # run describe (get statistics for the dataframe) with dask
    df_describe = df.describe().compute()

    # run groupby and count using dask
    df_grpby = df.groupby("VendorID").count().compute()

    context.log_dataset("describe", df=df_grpby, format="csv", index=True)
    return

In [10]:
# mlrun: end-code

## Test the function over Dask

### Load sample data

In [11]:
DATA_URL = "/User/examples/ytrip.csv"

In [12]:
!mkdir -p /User/examples/
!curl -L "https://s3.wasabisys.com/iguazio/data/Taxi/yellow_tripdata_2019-01_subset.csv" > {DATA_URL}

### Convert the code to MLRun function

Use `code_to_function` to convert the code to MLRun and specify the configuration for the Dask process (e.g. replicas, memory etc.). <br>
Note that the resource configurations are per worker.

In [13]:
# mlrun transforms the code above (up to nuclio: end-code cell) into serverless function
# which runs in k8s pods
fn = mlrun.code_to_function("test_dask", kind="job", handler="test_dask").apply(
    mlrun.mount_v3io()
)

### Run the function

In [14]:
# function URI is db://<project>/<name>
dask_uri = f"db://{project.name}/{dask_cluster_name}"

In [15]:
r = fn.run(
    handler=test_dask,
    inputs={"dataset": DATA_URL},
    params={"dask_function": dask_uri},
    auto_build=True,
)

## Track the progress in the UI

You can view the progress and detailed information in the MLRun UI by clicking on the uid above. <br>
To track the dask progress: in the Dask UI click the "dashboard link" above the "client" section.